<a href="https://colab.research.google.com/github/Noman-494/Gen-AI/blob/main/Financial_sentiment_anaysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

In [2]:
import pandas as pd
import numpy as np

# Train test split
from sklearn.model_selection import train_test_split

# Modeling
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Hugging Face Dataset
from datasets import Dataset

# Import accuracy_score to check performance
from sklearn.metrics import accuracy_score

In [3]:
df=pd.read_csv('/content/data.csv')
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5842 entries, 0 to 5841
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentence   5842 non-null   object
 1   Sentiment  5842 non-null   object
dtypes: object(2)
memory usage: 91.4+ KB


In [5]:
df.describe()

,Sentence,Sentiment
count,5842,5842
unique,5322,3
top,"Operating loss totalled EUR 0.9 mn , down from...",neutral
freq,2,3130


In [6]:
df.isnull().sum()

,0
Sentence,0
Sentiment,0


In [6]:
df.Sentiment.value_counts()

,count
Sentiment,
neutral,3130
positive,1852
negative,860


In [7]:
# prompt: neutral	3130
# positive	1852
# negative	860
#  convert into convert into numerical

from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
le = LabelEncoder()

# Fit the encoder on the 'Sentiment' column
df['Sentiment'] = le.fit_transform(df['Sentiment'])

print(df.Sentiment.value_counts())

Sentiment
1    3130
2    1852
0     860
Name: count, dtype: int64


In [8]:
df.Sentiment

,Sentiment
0,2
1,0
2,2
3,1
4,1
...,...
5837,0
5838,1
5839,1
5840,1


In [10]:
X=list(df.Sentence)
y=list(df.Sentiment)
y

[2,
 0,
 2,
 1,
 1,
 2,
 0,
 0,
 2,
 1,
 2,
 0,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 0,
 2,
 2,
 0,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 1,
 2,
 1,
 0,
 1,
 2,
 1,
 0,
 1,
 1,
 1,
 1,
 2,
 2,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 0,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 0,
 2,
 0,
 2,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 0,
 2,
 2,
 1,
 2,
 1,
 2,
 0,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 0,
 2,
 1,
 2,
 1,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 0,
 1,
 2,
 1,
 2,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 0,
 1,
 0,
 2,
 2,
 0,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 0,
 1,
 2,
 0,
 2,
 1,
 2,
 1,
 2,
 1,
 0,
 2,
 2,
 0,
 1,


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Check the number of records in training and testing dataset.
print(f'The training dataset has {len(X_train)} records.')
print(f'The testing dataset has {len(X_test)} records.')




The training dataset has 4673 records.
The testing dataset has 1169 records.


In [13]:
# Tokenizer from a pretrained model
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Tokenize the reviews
# X_train is already a list, no need to call to_list()
tokenized_data_train = tokenizer(X_train, return_tensors="np", padding=True)
tokenized_data_test = tokenizer(X_test, return_tensors="np", padding=True)

# Labels are one-dimensional numpy or tensorflow array of integers
labels_train = np.array(y_train)
labels_test = np.array(y_test)

# Tokenized ids
print(tokenized_data_train["input_ids"][0])

[  101  1109  1837  1298  1104  1103   162  9436 19093  1633  4854  1673
  1933  1209  1129   128   117  7208  4816   182  4859   117  5347  1568
  4816  2167  1105  1103  1459   112   188 10272  1298  1209  1703  1367
   117  4645  4816   182   119   138  1703  1512   119   122   110  1104
  1103  3380  1138  1151  1519   119   102     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0]


In [14]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile model using the string identifier "adam"
model.compile(optimizer="adam", loss=loss, metrics=['accuracy'])

In [17]:
model.fit(dict(tokenized_data_train),
          labels_train,
          validation_data=(dict(tokenized_data_test), labels_test),
          batch_size=4,
          epochs=2)

Epoch 1/2
1169/1169 [==============================] - 245s 169ms/step - loss: 1.1099 - accuracy: 0.4565 - val_loss: 1.1403 - val_accuracy: 0.5321
Epoch 2/2
1169/1169 [==============================] - 197s 168ms/step - loss: 1.0962 - accuracy: 0.4888 - val_loss: 1.0862 - val_accuracy: 0.3182


In [18]:
y_test_predict = model.predict(dict(tokenized_data_test))['logits']

# First 5 predictions
y_test_predict[:5]

37/37 [==============================] - 10s 200ms/step


array([[-0.62834275,  0.21710105,  0.6910832 ],
       [-0.62834275,  0.21710098,  0.69108325],
       [-0.62834287,  0.2171009 ,  0.69108325],
       [-0.62834287,  0.21710095,  0.6910832 ],
       [-0.6283428 ,  0.2171011 ,  0.69108325]], dtype=float32)

In [19]:
# Predicted probabilities
y_test_probabilities = tf.nn.softmax(y_test_predict)

# First 5 predicted probabilities
y_test_probabilities[:5]

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[0.14143702, 0.32940844, 0.52915454],
       [0.14143701, 0.3294084 , 0.52915454],
       [0.14143702, 0.32940838, 0.5291546 ],
       [0.14143701, 0.3294084 , 0.52915454],
       [0.14143701, 0.32940844, 0.52915454]], dtype=float32)>

In [20]:
# Predicted label
y_test_class_preds = np.argmax(y_test_probabilities, axis=1)

# First 5 predicted labels
y_test_class_preds[:5]

array([2, 2, 2, 2, 2])

In [21]:
accuracy_score(y_test_class_preds, y_test)

0.31822070145423437

In [22]:
# Save tokenizer
tokenizer.save_pretrained('./sentiment_transfer_learning_tensorflow/')

# Save model
model.save_pretrained('./sentiment_transfer_learning_tensorflow/')

In [23]:
tokenizer = AutoTokenizer.from_pretrained("./sentiment_transfer_learning_tensorflow/")

# Load model
loaded_model = TFAutoModelForSequenceClassification.from_pretrained('./sentiment_transfer_learning_tensorflow/')

Some layers from the model checkpoint at ./sentiment_transfer_learning_tensorflow/ were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ./sentiment_transfer_learning_tensorflow/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [24]:
y_test_predict = loaded_model.predict(dict(tokenized_data_test))['logits']

# Take a look at the first 5 predictions
y_test_predict[:5]

37/37 [==============================] - 13s 203ms/step


array([[-0.62834275,  0.21710105,  0.6910832 ],
       [-0.62834275,  0.21710098,  0.69108325],
       [-0.62834287,  0.2171009 ,  0.69108325],
       [-0.62834287,  0.21710095,  0.6910832 ],
       [-0.6283428 ,  0.2171011 ,  0.69108325]], dtype=float32)